# MSDR Analysis

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
import matplotlib.pyplot as plt
import pytz
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning

# Suppress specific statsmodels warnings
warnings.simplefilter('ignore', ValueWarning)
# Suppress ConvergenceWarning to avoid cluttering output during rolling window
warnings.simplefilter('ignore', ConvergenceWarning)
# Suppress RuntimeWarning from numpy which often happens during optimization of unstable models
warnings.simplefilter('ignore', RuntimeWarning)


In [ ]:
regional_emissions_final = {
    'f_hertz': pd.read_csv('../data/processed/analysis_final_f_hertz_15min_utc_202212312300_202412312245', sep=',', index_col=0),
    'amprion': pd.read_csv('../data/processed/analysis_final_amprion_15min_utc_202212312300_202412312245', sep=',', index_col=0),
    'tennet': pd.read_csv('../data/processed/analysis_final_tennet_15min_utc_202212312300_202412312245', sep=',', index_col=0),
    'transnetbw': pd.read_csv('../data/processed/analysis_final_transnet_bw_15min_utc_202212312300_202412312245', sep=',', index_col=0)
}

for df in regional_emissions_final.values():
    df.index = pd.to_datetime(df.index, format='ISO8601')
    df.sort_index(inplace=True)
    if df.index.tz is None:
        try:
            df.index = df.index.tz_localize('Europe/Berlin', ambiguous='infer')
        except pytz.exceptions.AmbiguousTimeError:
            df.index = df.index.tz_localize('Europe/Berlin', ambiguous=True)
        df.index = df.index.tz_convert('UTC')

    # Explicitly set frequency to 15 minutes
    df = df.asfreq('15min')


### Preparing Analysis Dataframe

In [ ]:
# Data inspection
print("I. DATA INSPECTION")
print("-"*20, "Checking for unwanted values", "-"*20)
for name, reg in regional_emissions_final.items():
    print(f"[{name}]")
    print(f"  1) Index data type: {reg.index.dtype}")
    print(f"  2) Duplicate Entries: {(reg.index.duplicated()).sum()}")
    print(f"  3) Negative Values for Generation: {(reg['total_generation'] < 0).sum()}")
    print(f"  4) Rows with NaN Values: {(reg.isnull().sum()).sum()}")


In [ ]:
# Data preparation
analysis_dfs = {}

print("II. DATA PREPARATION")
for name, reg in regional_emissions_final.items():
    print(f"[{name}]")
    print(f"  1) Calculation delta for time series:")

    # Calculating the delta between two consecutive rows to eliminate trends
    delta_reg = reg - reg.shift(1)
    delta_reg = delta_reg[1:] # Dropping the first row will be NaN

    # Ensure frequency is set after shifting/dropping
    delta_reg = delta_reg.asfreq('15min')

    # Fill any NaNs created by asfreq (if gaps existed) or shift
    if delta_reg.isnull().values.any():
        # Interpolate is often better than ffill for physical time series
        delta_reg = delta_reg.interpolate(method='time')
        # If NaNs remain (e.g. at start), drop them
        delta_reg = delta_reg.dropna()

    print(delta_reg.head(3))

    # Scaling data to have zero mean and unit variance (z-transformation)
    print(f"  2) Scaling data to have zero mean and unit variance:")
    scaler = StandardScaler()
    delta_reg[['total_generation', 'total_emissions']] = scaler.fit_transform(delta_reg[['total_generation', 'total_emissions']]) # Apply to cols to keep df
    print(delta_reg.head(3))

    # After transformation, check for missing values again
    print(f"  3) Rows with NaN Values after Transformation: {(delta_reg.isnull().sum()).sum()}")
    print("\n")

    # Add transformed dfs to dict to access later
    analysis_dfs[name] = delta_reg


### Performing MSDR Analysis

In [ ]:
# Defining model params and vars

# Using a rolling time window to allow for the model to adjust over time
win_len = 7*24*4 # size of the time window (1 week = 672 values for 15-min data)

# Simplified Parameter Grid
param_grid = {
    'k_regimes': [2, 3],                 # Number of regimes
    'switching_variance': [True, False], # Whether variance switches between regimes
    'trend': ['c', 't'],                 # c = intercept, t = time trend
}

def fit_markov_model(ret_window, params):
    try:
        # We always use total_generation as exogenous variable to determine MEF
        msdr_model = sm.tsa.MarkovRegression(
            endog=ret_window['total_emissions'],
            k_regimes=params['k_regimes'],
            trend=params['trend'],
            exog=ret_window[['total_generation']], # Always include Generation
            switching_variance=params['switching_variance']
        )

        # Increased maxiter to help convergence
        msdr_result = msdr_model.fit(disp=False, maxiter=200)

        # Predict within sample to calculate error
        msdr_predict = msdr_result.predict(start=ret_window.index[0], end=ret_window.index[-1])

        # Compute MAE (Mean Absolute Error)
        mae = np.mean(np.abs(ret_window['total_emissions'] - msdr_predict))

        return msdr_result, msdr_result.aic, mae

    except Exception as e:
        # print(f"Model fitting failed for parameters: {params} with error: {e}")
        return None, np.inf, np.inf

# Function to process one window
def process_window(i, data, grid):
    roll_win = data.iloc[i:i+win_len]

    # Ensure window has frequency set
    if roll_win.index.freq is None:
        roll_win = roll_win.asfreq('15min')

    best_result = None
    best_mae = np.inf

    for params in ParameterGrid(grid):
        result, aic, mae = fit_markov_model(roll_win, params)
        if mae < best_mae:
            best_result = result
            best_mae = mae

    return best_result

# Select which region to analyze (e.g., 'tennet') or loop over them
current_region_name = 'tennet'
current_data = analysis_dfs[current_region_name]

print(f"Starting analysis for {current_region_name} with {len(current_data)} rows...")

# Run Parallel execution
results = Parallel(n_jobs=-1)(
    delayed(process_window)(i, current_data, param_grid)
    for i in range(len(current_data) - win_len + 1)
)

print("Analysis complete.")


In [ ]:
# Estimate the emission time series using the best model for each window
estimated_emi = current_data[['total_emissions']].copy()
estimated_emi['estimated_emissions'] = np.nan

for i in range(len(current_data)-win_len+1):
    reg_win = current_data.iloc[i:i+win_len]
    msdr_results = results[i]

    if msdr_results is not None:
        forecast = msdr_results.predict(start=reg_win.index[-1], end=reg_win.index[-1])
        estimated_emi.loc[reg_win.index[-1], 'estimated_emissions'] = forecast[0]
    else:
        pass

print("Estimation complete. Head of results:")
print(estimated_emi.head())
